### Author : Si Dan HO

# Calibration et pricing des dérivés de taux d'intérêt

In [1]:
import time
import datetime as dt
import pandas as pd
import numpy as np
import math
import scipy.stats as sps
import scipy as sp
import scipy.optimize as spop
import matplotlib.pyplot as plt
import os

import pylab as py

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib as mpl

import random

## Réplication et arbitrage

Le but de ce paragraphe est de retrouver l'expression du taux linéaire forward à chaque instant via un raisonnement d'arbitrage. il convient de rappeler qu'un contrat forward sur un actif sous-jacent $S$ est un produit dérivé permettant à deux agents de s'échanger ce sous-jacent (respectivement achat et vente) à une date $T$ fixé et à un prix fixé (qui est le prix forward du sous-jacent) à la signature du contrat à l'instant $0$.

Par ailleurs, le juste prix de ce contrat à chaque instant doit respecter le principe d'absence d'opportunité d'arbitrage. Cela signifie qu'il n'est aucunement possible d'obtenir un gain strictement positif à partir d'un investissement nul. Plus formellement, il y a arbitrage entre les instants $0$ et $T$ pour un portefeuille autofinançant $X$ de prix $X_t$ à l'instant $t$ si:

- Sa condition initiale est nulle : $X_0 = 0$
- $\mathbb{P}[X_T \geq 0] = 1$ et $\mathbb{P}[X_T > 0] > 0$

Il convient de noter que les hypothèses d'efficience des marchés, garantissant un prix unique, ainsi que la complétude de ceux ci, permettant la réplication de l'ensemble des flux par des instruments de marché liquides, doivent être respectées dans le cadre de la valorisation au juste prix.

Sachant que le prix d'un instrument financier est le coût de sa réplication par les instruments de marché faisant l'objet d'un échange et/ou d'une cotation, il est possible de déterminer celui d'un contrat forward par un raisonnement d'arbitrage.

Ce raisonnement permet d'établir qu'en l'absence d'opportunité d'arbitrage, si l'on considère $X$ et $Y$ deux portefeuilles autofinançants, alors leur valeur sont telle que :

$$
X_T = Y_T \ \ \mathbb{P} \ p.s. \ \Longrightarrow X_t = Y_t \ \ \mathbb{P} \ p.s. \ \forall t \in [0,T]
$$

De ce fait, à l'instant $t=0$, on considère un premier portefeuille où l'on contracte un contrat forward sur un sous-jacent $S$. De ce fait, les deux parties s'engagent à s'échanger à l'instant $T$ ce sous-jacent à un prix $F_0(T)$ fixé. Il n'y a aucun échange de flux à cet instant $0$. De ce fait, la valeur de ce contrat pour l'acheteur à maturité est égale à $S_T - F_0(T)$.

Par ailleurs, on considère un second portefeuille où la stratégie suivante est adoptée à l'instant $0$:

- Acheter une unité du sous-jacent de prix $S_0$
- Vendre une quantité $F_0(T)$ d'obligations zéro coupon. Le flux lié à cette transaction est donné par : $-F_0(T)B(0,T)$

Ainsi, la valeur totale de ce portefeuille est :

- à $t=0$ : $S_0-F_0(T)B(0,T)$
- à maturité $T$ : $S_T-F_0(T)$ par désactualisation

L'absence d'opportunité d'arbitrage stipule que sachant que la valeur de ces deux portefeuilles $X$ et $Y$ à maturité $T$ sont égaux à $S_T-F_0(T)$. Alors leur prix actuel à $t=0$ doivent tout aussi être égaux. Cela donne ainsi l'équation :

$$
0 = S_0-F_0(T)B(0,T)
$$

Ainsi, le prix forward du sous-jacent $S$ est donné par :

$$
F_0(T) = \frac{S_0}{B(0,T)}
$$

Et plus généralement à tout instant $t \in [0,T]$, ce prix forward est :

$$
F_t(T) = \frac{S_t}{B(t,T)}
$$

En particulier, le montant à investir en $T$ pour se garantir d'un euro payé en $T+\delta$ peut être évalué en $t$ comme
le prix d'un contrat forward sur le zéro-coupon nommé $B_t(T,T+\delta)$, égal à $\frac{B(t,T+\delta)}{B(t,T)}$. 

Rappelons que le taux linéaire est le taux $L(t,T)$ telle que pour un zéro coupon $B(t,T)$ :

$$
B(t,T) = \frac{1}{1+(T-t)L(t,T)}
$$

Ce taux associé à un contrat forward sur le zéro-coupon est $L_t(T,T+\delta)$, et est nommé taux forward linéaire, c'est à dire le taux "vu" à l'instant $T$ pour des prêts zéro-coupons de ténor $\delta$ remboursant un euro à maturité.

Ce taux forward est donc tel que :

$$
B_t(T,T+\delta) = \frac{1}{1+\delta L_t(T,T+\delta)} = \frac{B(t,T+\delta)}{B(t,T)}
$$

De ce fait, la valeur du taux linéaire forward à chaque instant $t \in [0,T]$ est donné par :

$$
L_t(T,T+\delta) = \frac{1}{\delta}(\frac{B(t,T)}{B(t,T+\delta)} - 1)
$$

## Modèle de Black

### Valorisation d'un Caplet 

Il existe plusieurs produits dérivés de taux d'intérêts. L'un des plus connus est le caplet, un produit dérivé vanille pouvant être considéré comme une option d'achat (ou Call) dont le sous-jacent est un taux d'intérêt. Ainsi le détenteur d'un caplet peut se prémunir d'une hausse de taux d'intérêt et obtenir in fine un payoff à l'instant $T+\delta$ lorsque le taux variable $L(T,T+ \delta)$ qui prévaudra à cet instant $T$ pour des prêts de maturité $\delta$ excède le Strike $K$ fixé, en ayant le droit d'emprunter ou de prêter à ce taux fixe $K$ garanti, qui est souvent proche du taux forward (d'après Huyen Pham). Ainsi, le payoff d'un caplet est donné par :

$$
\Pi_{Caplet}(T,\delta,K) = \delta(L(T, T+\delta) - K)_{+}
$$

Et en l'absence d'opportunité d'arbitrage, si l'on pose $\mathbb{Q}$ la probabilité risque neutre et $r_t$ le taux sans risque, le théorème fondamental de la valorisation des actifs permet d'établir que le prix actuel d'un caplet est donné par :

$$
Caplet(T,\delta,K) = \delta \mathbb{E}^{\mathbb{Q}}[e^{\int_0^{T+\delta} r_s ds}(L(T, T+\delta) - K)_{+}]
$$

Si l'on pose $L_f(t,T,T+\delta)$ le taux forward à l'instant $t$ expirant à l'instant $T$ et de maturité $T+ \delta$, celui ci est tel que :

$$
L_f(t,T,T+\delta) = \frac{1}{\delta}(\frac{B(t,T)}{B(t,T+\delta)} -1)
$$

de sorte à pouvoir introduire la probabilité $T+\delta$ forward neutre, écrite $\mathbb{Q}^T$ équivalente à la probabilité risque neutre $\mathbb{Q}$, définie telle que :

$$
\frac{d\mathbb{Q}^{T+\delta}}{d\mathbb{Q}}= Z_t^{T+\delta} = \frac{e^{\int_t^{T+\delta} r_s ds}}{\mathbb{E}^{\mathbb{Q}}[e^{\int_t^{T+\delta} r_s ds}|\mathcal{F}_t]}
$$

En convenant de remarquer que : $\mathbb{E}^{\mathbb{Q}}[Z_t^{T+\delta}|\mathcal{F}_t] = 1$

Il convient de préciser par ailleurs que cette probabilité forward neutre est associée au numéraire $B(t,T+\delta)$, ce qui signifie qu'il est possible d'établir que pour un certain actif $X_t$, le processus $\frac{X_t}{B(t,T+\delta)}$ est une martingale sous cette probabilité :

$$
\frac{X_t}{B(t,T+\delta)} = \mathbb{E}^{\mathbb{Q}^{T+\delta}}[\frac{X_T}{B(T,T+\delta)}|\mathcal{F}_t]
$$


En particulier, il est possible de constater que l'expression du taux forward à l'instant $t$ est exprimé comme suit :

$$
L_f(t,T,T+\delta) = \frac{1}{\delta}(\frac{B(t,T)}{B(t,T+\delta)} -1) = \frac{1}{\delta}\frac{B(t,T)}{B(t,T+\delta)} -\frac{1}{\delta}
$$


Or, pour un contrat forward sur un zéro coupon de maturité $T$ pour un prêt de 1 euros d'une durée de $T+\delta$, il est possible de constater que l'inverse de son prix à l'instant $t$ est exprimé comme suit, grâce au changement de numéraire :

$$
\frac{1}{B_f(t,T,T+\delta)} = \frac{B(t,T)}{B(t,T+\delta)} = \mathbb{E}^{\mathbb{Q}^{T+\delta}}[\frac{B(T,T)}{B(T,T+\delta)}|\mathcal{F}_t] = \mathbb{E}^{\mathbb{Q}^{T+\delta}}[\frac{1}{B(T,T+\delta)}|\mathcal{F}_t]
$$

Ainsi, le taux forward à l'instant $t$ est égal à :

$$
L_f(t,T,T+\delta) = \frac{1}{\delta}\mathbb{E}^{\mathbb{Q}^{T+\delta}}[\frac{1}{B(T,T+\delta)}|\mathcal{F}_t] -\frac{1}{\delta} = \frac{1}{\delta}\mathbb{E}^{\mathbb{Q}^{T+\delta}}[\frac{1}{B(T,T+\delta)} - 1|\mathcal{F}_t] = \mathbb{E}^{\mathbb{Q}^{T+\delta}}[L(T,T+\delta)|\mathcal{F}_t]
$$

avec $L(T,T+\delta)$ le taux interbancaire prévalent à l'instant $T$ pour des prêts de durée $\delta$. Ainsi, la probabilité $T+\delta$ forward neutre est définie de sorte que le taux forward interbancaire $L_f(t,T,T+\delta)$ soit une martingale sous cette probabilité.

Il a été vu en début de partie que le prix actuel d'un caplet de strike $K$, de maturité $T$ et de ténor $\delta$ est donné par :

$$
Caplet(T,\delta,K) = \delta \mathbb{E}^{\mathbb{Q}}[e^{\int_0^{T+\delta} r_s ds}(L(T, T+\delta) - K)_{+}] = \delta B(0,T+\delta)\mathbb{E}^{\mathbb{Q}}[\frac{e^{\int_0^{T+\delta} r_s ds}}{B(0,T+\delta)}(L(T, T+\delta) - K)_{+}]
$$

Or il a été vu précédemment que la probabilité $T+\delta$ forward neutre était construite par rapport à la probabilité risque neutre de telle sorte que pour tout $t \in [0,T]$ :

$$
\frac{d\mathbb{Q}^{T+\delta}}{d\mathbb{Q}}= Z_t^{T+\delta} = \frac{e^{\int_t^{T+\delta} r_s ds}}{\mathbb{E}^{\mathbb{Q}}[e^{\int_t^{T+\delta} r_s ds}|\mathcal{F}_t]} =  \frac{e^{\int_t^{T+\delta} r_s ds}}{B(t,T+\delta)}
$$

Dans le cas où l'on souhaite valoriser selon l'instant 0, 

$$
\frac{d\mathbb{Q}^{T+\delta}}{d\mathbb{Q}}= Z_0^{T+\delta} = \frac{e^{\int_0^{T+\delta} r_s ds}}{\mathbb{E}^{\mathbb{Q}}[e^{\int_0^{T+\delta} r_s ds}|\mathcal{F}_t]} =  \frac{e^{\int_0^{T+\delta} r_s ds}}{B(0,T+\delta)}
$$


De ce fait, par un changement de numéraire selon la probabilité $T+\delta$ forward neutre, ce prix devient :

$$
Caplet(T,\delta,K) = \delta B(0,T+\delta) \mathbb{E}^{\mathbb{Q}^{T+\delta}}[(L(T, T+\delta) - K)_{+}]
$$

Sachant que le taux forward interbancaire $L(t,T, T+\delta)$, que l'on nommera simplement $L_t$, est une martingale sous la probabilité $T+\delta$ forward neutre, alors en posant le $\mathbb{Q}^{T+\delta}$ mouvement brownien $W^{\mathbb{Q}^{T+\delta}}_t$ et en admettant la volatilité du taux forward constante par rapport au temps comme c'est le cas dans l'univers de Black, sa dynamique est donnée par :

$$
\frac{dL_t}{L_t} = \sigma(T,\delta,K)dW^{\mathbb{Q}^{T+\delta}}_t
$$

Ainsi, l'expression du taux forward $L_t$ est :

$$
L_t = L_0exp(\sigma(T,\delta,K)W^{\mathbb{Q}^{T+\delta}}_t - \frac{1}{2}\sigma^2(T,\delta,K)t)
$$

Sachant que $W^{\mathbb{Q}^{T+\delta}}_t$ suit une loi normale $\mathcal{N}(0,t)$ sous la probabilité $T+\delta$ forward neutre, alors en posant la variable $Z$ gaussienne centrée réduite :

$$
L_t = L_0exp(\sigma(T,\delta,K)\sqrt{t}Z - \frac{1}{2}\sigma^2(T,\delta,K)t)
$$

Et le prix actuel d'un caplet est donné sous la probabilité historique par:

$$
Caplet(T,\delta,K) = \delta B(0,T+\delta) \mathbb{E}[(L_0e^{\sigma(T,\delta,K)\sqrt{T}Z - \frac{1}{2}\sigma^2(T,\delta,K)T} - K)_{+}]
$$

Le calcul analytique de cette espérance, fortement analogue au raisonnement effectué pour le modèle de Black Scholes, donne :

$$
Caplet(T,\delta,K) = \delta B(0,T+\delta)(P_1 - P_2)
$$

avec :

- $P_1 = \mathbb{E}[L_0e^{\sigma(T,\delta,K)\sqrt{T}Z - \frac{1}{2}\sigma^2(T,\delta,K)T} \mathbb{1}_{L_0e^{\sigma(T,\delta,K)\sqrt{T}Z - \frac{1}{2}\sigma^2(T,\delta,K)T} \geq K}]$
- $P_2 = K \mathbb{E}[\mathbb{1}_{L_0e^{\sigma(T,\delta,K)\sqrt{T}Z - \frac{1}{2}\sigma^2(T,\delta,K)T} \geq K}] = K \mathbb{P}[L_0e^{\sigma(T,\delta,K)\sqrt{T}Z - \frac{1}{2}\sigma^2(T,\delta,K)T} \geq K]$

Ainsi, si l'on pose : $d = \frac{1}{v}(ln(\frac{L_0}{K}) +\frac{v^2}{2})$ avec $v =\sigma(T,\delta,K)\sqrt{T}$, alors :

$$
P_2 = K\mathcal{N}(d-v)
$$

Concernant la quantité $P_1$, celle-ci se calcule de la manière suivante , et grâce à un changement de variable intermédiaire :

$$
P_1 = \frac{L_0}{2\pi}\int_{L_0e^{\sigma(T,\delta,K)\sqrt{T}z - \frac{1}{2}\sigma^2(T,\delta,K)T} \geq K} e^{\sigma(T,\delta,K)\sqrt{T}z - \frac{1}{2}\sigma^2(T,\delta,K)T} e^{-\frac{z^2}{2}} dz = \frac{L_0}{2\pi}\int_{L_0e^{\sigma(T,\delta,K)\sqrt{T}z + \frac{1}{2}\sigma^2(T,\delta,K)T} \geq K} e^{-\frac{z^2}{2}} dz = L_0\mathcal{N}(d)
$$

C'est ainsi que le prix actuel d'un caplet de maturité $T$, de ténor $\delta$ et de Strike $K$ est donnée par la formule de Black :

$$
Caplet(T,\delta,K) = \delta B(0,T+\delta)(L_0\mathcal{N}(d)- K\mathcal{N}(d-v))
$$

En se rappelant que $L_0 = L(0,T,T+\delta)$ est le taux actuel forward de maturité $T$ et de ténor $\delta$. Ainsi, le modèle de Black pour valoriser un caplet peut être numériquement implémenté comme suit :

In [ ]:
def BlackCaplet(T,delta,sigma,B0, L0, K) :
    v = sigma*math.sqrt(T)
    d = (1/v)*(math.log(L0/K) - 0.5*(v**2))
    Caplet = delta*B0*(L0*sps.norm.cdf(d) - K*sps.norm.cdf(d-v))
    return Caplet

Considérons de ce fait un caplet EURIBOR 6 mois de maturité 1 an sachant que le marché côte une volatilité Black de 30%. Son prix actuel est donc calculé, sachant que de ce fait,

- $T = 1$
- $\delta = 6m = 0.5$
- $\sigma = 30\%$

### Valorisation d'une Swaption Payeuse

Supposons une swaption payeuse débutant au temps $T$ et de maturité $T_n$ c'est à dire de ténor $T_n - T$. le coupon c est le taux fixe.

Premièrement, le prix d'un swap de taux Le swap payeuse de taux fixe, de nominal 1 et de strike $K$ sur la période d’investissement $(T_0,T_n)$ au temps t est, d'après le théorème fondamental de la valorisation des actifs et sachant les flux futurs du produit: 

$$
P_{Swap}(t,T_0,T_n,K) = \sum_{i=1}^{n}B(t,T_i)(T_i - T_{i-1})[L(t,T_{i-1}, T_i) - K]
$$

où $L(t,T_{i-1}, T_i)$ est le taux forward, c'est à dire taux établi au temps $t$ et qui est appliquable sur la période $[T_{i-1}, T_i]$. Il s'agit par ailleurs du taux permettant de valoriser un Forward Rate Agreement (FRA) au temps $t$ à sa Juste Valeur selon l'absence d'opportunité d'arbitrage (le prix doit être nul au temps $t$). L'expression de ce taux Forward est :

$$
L(t,T_{i-1}, T_i) = \frac{1}{T_{i}-T_{i-1}}(\frac{B(t, T_{i-1})}{B(t, T_i)} - 1)
$$

De ce fait, en réinjectant cette formule dans le prix du swap, il s'en suit que :

$$
P_{Swap}(t,T_0,T_n,K)  = B(t,T_0) - B(t,T_n) - \sum_{i=1}^{n} KB(t,T_i)(T_i - T_{i-1})
$$

cela permet ainsi de trouver le taux dit Swap Forward $\omega_t$ qui est la valeur de $K$ permettant au contrat swap d'être à son Juste Prix, c'est à dire nul au temps $t$. Son expression est donné par :

$$
\omega_t = \frac{B(t,T_0) - B(t,T_n)}{lvl(t,T_n)}
$$

Avec

$$
lvl(t,T_n) =\sum_{i=0}^{n}(T_i - T_{i-1})B(t,T_i)
$$

Cette nouvelle quantité permet ainsi d'obtenir l'expression nouvelle du swap :

$$
P_{Swap}(t,T_0,T_n,K)= \omega_tlvl(t,T_n) - K lvl(t,T_n) = (\omega_t - K)lvl(t, T_n)
$$

Le Swaption quant à elle, est une option de Swap. Elle donne donc le droit, et non l'obligation, au détenteur d'exercer son swap (receveur de taux fixe dans ce projet). Plus précisément, une swaption payeuse est une option permettant d’entrer dans un swap payeur applicable au temps $T_0$, et prenant fin au temps $T_n$ (et donc de ténor $T_n - T_0$, de nominal ici égal à 1 et de strike K. Ainsi, $T_0$ est la maturité de l'option.

Ainsi, son cash-flow à maturité est simplement le maximum entre le prix du swap receveur de taux fixe évalué au temps $T_0$ et la valeur nulle. 

De ce fait, son prix au temps $t$ est donnée par  l'espérance de son cash-flow futur actualisé par le taux sans risque $r_t$, et donc :

$$
Swopt^{PAY}(t,T_0,T_n,K)=\mathbb{E}^{\mathbb{Q}}[e^{-\int_t^{T_0} r_udu}(\omega_{T_0} - K)_{+}lvl(T_0, T_n)]
$$

Ainsi donc, à l'instant $0$ le prix devient :

$$
Swopt^{PAY}(0,T_0,T_n,K)=\mathbb{E}^{\mathbb{Q}}[e^{-\int_0^{T_0} r_udu}(\omega_{T_0} - K)_{+}lvl(T_0, T_n)]
$$

Il est possible d'effectuer un changement de numéraire selon la probabilité Duration $\mathbb{Q}^{lvl}$ et de constater que le processus $\omega_t$ est une martingale sous cette probabilité. Sachant que pour un processus $X_t$, la formule de pricing est la suivante :

$$
X_t = lvl(t, T_n)\mathbb{E}^{\mathbb{Q}^{lvl}}[\frac{X_T}{lvl(T, T_n)}|F_t]
$$

Nous pouvons établir que le prix actuel du swaption est :

$$
Swopt^{PAY}(0,T_0,T_n,K)=lvl(0, T_n)\mathbb{E}^{\mathbb{Q}^{lvl}}[\frac{P_{Swaption}(T_0,T_0,T_n,K)}{lvl(T_0, T_n)}|F_0]
$$

Et étant donné que son prix à sa maturité $T_0$ est égal à son cash flow (qui est déterministe) et donc égal à :

$$
Swopt^{PAY}(T_0,T_0,T_n,K) = (\omega_{T_0} - K)_{+}lvl(T_0, T_n)
$$

Alors le prix final du Swaption est égal à :

$$
Swopt^{PAY}(0,T_0,T_n,K)= lvl(0, T_n)\mathbb{E}^{\mathbb{Q}^{lvl}}[(\omega_{T_0} - K)_{+}]
$$

Il est possible de calculer directement la quantité $\mathbb{E}^{\mathbb{Q}^{lvl}}[(\omega_{T_0} - K)_{+}]$ pour arriver au modèle de Black.

En effet, sachant que $\omega_t = \frac{B(t,T_0) - B(t,T_n)}{lvl(t,T_n)}$ est une martingale sous la probabilité Duration $\mathbb{Q}^{lvl}$, alors en supposant sa volatilité constante par rapport au temps $t$, mais exprimée en fonction du Strike $K$, de la maturité $T_0$ et du ténor $T_n - T_0$ (volatilité qui est aussi celle côtée sur le marché), sa dynamique sous cette probabilité est donnée par :

$$
\frac{d\omega_t}{\omega_t} = \sigma(T_0,T_n,K)dW^{\mathbb{Q}^{lvl}}_t
$$

d'où son expression à l'instant $T_0$:

$$
\omega_{T_0} = \omega_0exp(\sigma(T_0,T_n,K)dW^{\mathbb{Q}^{lvl}}_{T_0} - \frac{1}{2}\sigma^2(T_0,T_n,K) T_0)
$$

Et le prix d'une swaption payeuse devient :

$$
Swopt^{PAY}(0,T_0,T_n,K)= lvl(0, T_n)\mathbb{E}^{\mathbb{Q}^{lvl}}[(\omega_0e^{\sigma(T_0,T_n,K)dW^{\mathbb{Q}^{lvl}}_{T_0} - \frac{1}{2}\sigma^2(T_0,T_n,K) T_0} - K)_{+}]
$$

Et un calcul analogue d'espérance que celui effectué pour la valorisation d'un caplet permet ainsi d'établir la formule de Black dans le cas d'une swaption payeuse :

$$
Swopt^{PAY}(0,T_0,T_n,K) = lvl(0, T_n)(\omega_0\mathcal{N}(d)- K\mathcal{N}(d-v))
$$

Avec :$d = \frac{1}{v}(ln(\frac{\omega_0}{K}) +\frac{v^2}{2})$ avec $v =\sigma(T_0,T_n,K)\sqrt{T_0}$.

Ainsi, le modèle de Black pour la valorisation d'une Swaption Payeuse peut être numériquement implémenté comme suit :

In [ ]:
def BlackSwopt(T,delta,sigma,lvl0, w0, K) :
    v = sigma*math.sqrt(T)
    d = (1/v)*(math.log(w0/K) - 0.5*(v**2))
    Caplet = lvl0*(L0*sps.norm.cdf(d) - K*sps.norm.cdf(d-v))
    return Caplet

Considérons de ce fait un un swap vanille EURIBOR 6M 5 ans annulable dans 3 ans (le payeur de taux fixe a le droit d'arrêter sans frais le swap dans trois ans). La volatilité implicite est côtée à 30%. Son prix actuel est donc calculé :

## Cadre de Heath-Jarrow-Morton (HJM) et Volatilité de Ho & Lee

### Cadre de Heath-Jarrow-Morton

Le framework de Heath-Jarrow-Morton spécifie la dynamique des taux forward $f(t,T)$ via leur volatilité locale de la manière suivante, en posant $\alpha(t,T)$ et $\sigma(t,T)$ des processus $\mathcal{F}_t$ adaptés :

$$
\left\{
    \begin{array}{ll}
    df(t,T) = \alpha(t,T)dt + \sigma(t,T)dW_t \\
    f(0,T) = f^{m}(0,T)
    \end{array}
\right.
$$

où $f^{m}(0,T)$ est le taux forward observé sur le marché. 

Par ailleurs, le taux forward peut s'exprimer en fonction du prix d'une obligation zéro coupon comme suit :

$$
f(t,T) = -\frac{\partial ln B(t,T)}{\partial T}
$$

De ce fait, cela permet d'établir le prix d'une obligation zéro-coupon de la manière suivante :

$$
B(t,T) = exp(-\int_t^T f(t,u)du)
$$

Ainsi, sachant par ailleurs que le taux court est égal à : $r_t = f(t,t)$, d'après la règle de Leibniz appliquée à la fonction $F(t,T) = -\int_t^T f(t,u)du$ permet d'établir que :

$$
dF(t,T) = f(t,t)dt -\int_t^T df(t,u)du = r_tdt -\int_t^T \alpha(t,u)dudt - \int_t^T \sigma(t,u)dudW_t
$$

En posant :

- $A(t,T) = \int_t^T \alpha(t,u)dudt$
- $\Sigma(t,T) = \int_t^T \sigma(t,u)du$

la dynamique devient : $dF(t,T) = r_tdt - A(t,T)dt - \Sigma(t,T)dW_t$
Ainsi, étant donné que $B(t,T) = exp(F(t,T))$, alors la formule d'Itô appliquée à la fonction exponentielle donne la dynamique suivante du zéro-coupon :

$$
dB(t,T) = B(t,T)[dF(t,T) - \frac{1}{2} d<F(.,T)>_t]
$$

d'où :

$$
dB(t,T) = B(t,T)[(r_t -A(t,T) + \frac{1}{2} \Sigma(t,T)^2) dt - \Sigma(t,T)dW_t]
$$

En émettant l'hypothèse d'absence d'opportunité d'arbitrage, il existe une probabilité risque neutre équivalente à la probabilité historique auquelle les prix actualisés des actifs sont des martingales. 

Or, en considérant $r_t$ le taux sans risque, la dynamique du prix actualisé des obligations zéro coupons sont :

$$
d(e^{-\int_t^T r_udu} B(t,T)) = e^{-\int_t^T r_udu}B(t,T)[(- A(t,T) + \frac{1}{2} \Sigma(t,T)^2) dt - \Sigma(t,T)dW_t]
$$

Si l'on pose $\theta(t)$ de sorte que :

$$
-\Sigma(t,T) \theta(t) = - A(t,T) + \frac{1}{2} \Sigma(t,T)^2
$$ 

et que la variable $L_T = exp(\int_0^T \theta(s)dW_t = - \frac{1}{2}\int_0^T \theta^2(s) ds)$, d'espérance 1, puisse définir une nouvelle probabilité $\mathbb{Q}$ sous la condition de Novikov (qui est ici admise), alors d'après le théorème de Girsanov, le processus $W^{\mathbb{Q}}_t = W_t - \int_0^t \theta(s)ds$ est un mouvement brownien sous $\mathbb{Q}$.

De ce fait, sous cette nouvelle probabilité $\mathbb{Q}$, la dynamique du prix actualisé des obligations zéro coupons sous $\mathbb{Q}$ devient :

$$
d(e^{-\int_t^T r_udu} B(t,T)) = -\Sigma(t,T) e^{-\int_t^T r_udu}B(t,T)(dW_t - \theta(t)dt) = \Sigma(t,T) e^{-\int_t^T r_udu}B(t,T)dW^{\mathbb{Q}}_t
$$

Et le prix actualisé des obligations zéro coupons sont bien des martingales sous cette probabilité $\mathbb{Q}$. Ainsi, cette probabilité nouvellement construite via le processus $(\theta(t))_t$ est bien une probabilité risque neutre, montrant que s'il existe un processus $(\theta(t))_t$ tel que :

$$
-\Sigma(t,T) \theta(t) = - A(t,T) + \frac{1}{2} \Sigma(t,T)^2
$$

Alors le modèle de Heath-Jarrow-Morton vérifie bien la condition d'absence d'opportunité d'arbitrage. Et la dynamique d'une obligation zéro coupon sous la probabilité risque neutre est donnée par :

$$
dB(t,T) = B(t,T)(r_t dt - \Sigma(t,T) dW^{\mathbb{Q}}_t)
$$

Par ailleurs, il est possible de réécrire la dynamique des taux forward $f(t,T)$ sous la probabilité risque neutre. Par la formule d'Itô appliquée à $lnB(t,T)$, il est possible d'obtenir :

$$
dlnB(t,T) = \frac{1}{B(t,T)}dB(t,T) - \frac{1}{2B(t,T)^2} d<B(t,T)>
$$

d'où :

$$
dlnB(t,T) = \frac{1}{B(t,T)}dB(t,T) - \frac{1}{2B(t,T)^2} d<B(t,T)> = (r_t - \frac{1}{2}(\Sigma(t,T))^2)dt - \Sigma(t,T) dW^{\mathbb{Q}}_t
$$

en intégrant entre 0 et $t$ nous avons :

$$
lnB(t,T) = lnB(0,T)  + \int_0^t (r_s - \frac{1}{2}(\Sigma(s,T))^2)ds - \int_0^t \Sigma(s,T) dW^{\mathbb{Q}}_s
$$

Or : $f(t,T) = -\frac{\partial ln B(t,T)}{\partial T}$. Ainsi, admettant qu'il est possible d'inverser la dérivée par rapport à $T$ et l'intégrale grâce à un théorème d'Intégration, il vient que :

$$
f(t,T) = \int_0^t \Sigma(s,T) \frac{\partial \Sigma(s,T)}{\partial T} ds + \int_0^t \frac{\partial \Sigma(s,T)}{\partial T} dW^{\mathbb{Q}}_s
$$

De ce fait, la dynamique du taux forward $f(t,T)$ est égale à :

$$
df(t,T)  = \frac{\partial \Sigma(t,T)}{\partial T}\Sigma(t,T)dt + \frac{\partial \Sigma(t,T)}{\partial T} dW^{\mathbb{Q}}_t
$$

Sachant que : $\Sigma(t,T) = \int_t^T \sigma(t,u)du$, alors $\frac{\partial \Sigma(t,T)}{\partial T} = \sigma(t,T)$ et la dynamique devient :

$$
df(t,T) = \sigma(t,T)\Sigma(t,T)dt + \sigma(t,T) dW^{\mathbb{Q}}_t
$$

En conclusion, le drift du taux forward sous laquelle l'absence d'opportunité d'arbitrage est respectée dépend entièrement de la volatilité $\sigma(t,T)$ et est égale à :

$$
\alpha(t,T) = \sigma(t,T)\Sigma(t,T) = \sigma(t,T)\int_t^T \sigma(t,u)du
$$

### Volatilité de Ho & Lee

Etant donné que le cadre de Heath-Jarrow-Morton est vaster et qu'il existe une multitude de modèle, il convient de spécifier précisément la forme de la volatilité $\sigma(t,T)$ du taux forward $f(t,T)$ à partir d'un certain modèle. De ce fait, l'un des modèles les plus connues est celle établie par Ho & Lee, qui suppose tout simplement que cette volatilité est constante :

$$
\sigma(t,T) = \sigma
$$

De ce fait, $\Sigma(t,T) = \sigma^2 (T-t)$ et la dynamique du taux forward est donnée par :

$$
df(t,T) = \sigma^2 (T-t) + \sigma dW^{\mathbb{Q}}_t
$$

d'où par intégration entre 0 et $t$ :

$$
f(t,T) = f(0,T) + \sigma^2 t(T - \frac{t}{2}) + \sigma W^{\mathbb{Q}}_t
$$

Sachant que le taux court s'écrit $r_t = f(t,t)$, alors son expression est donnée par :

$$
r_t = f(t,t) = f(0,t) + \sigma^2 \frac{t^2}{2} + \sigma W^{\mathbb{Q}}_t
$$

Ainsi, en posant $X_t = r_t - f(0,t) = - \sigma^2 \frac{t^2}{2} + \sigma W^{\mathbb{Q}}_t$, par une simple application de la formule d'Itô, la dynamique de ce processus est égale à :

$$
dX_t = \sigma^2 tdt + \sigma dW^{\mathbb{Q}}_t
$$

et en réinjectant $X_t$ dans l'expression du taux forward $f(t,T)$, il vient que :

$$
f(t,T) = f(0,T) + X_t + \sigma^2t(T-t)
$$

En intégrant par rapport à la deuxième variable entre $t$ et $T$, sachant que le processus $X_t$ ne dépend pas de cette variable $T$, il est possible de constater que :

$$
\int_t^T f(t,u)du = \int_t^T f(0,u)du + X_t (T-t) + {\sigma^2}{2}t(T-t)^2 = \int_0^T f(0,u) - \int_0^t f(0,u) + X_t (T-t) + {\sigma^2}{2}t(T-t)^2
$$

Ainsi, sachant que le prix d'une obligation zéro coupon s'exprime en fonction du taux forward de la manière suivante :

$$
B(t,T) = exp(-\int_t^T f(t,s)ds)
$$,

Alors sachant que $exp(\int_0^T f(0,s)ds - \int_0^t f(0,s)ds) = \frac{exp(\int_0^T f(0,s)ds)}{exp(\int_0^t f(0,s)ds)} = \frac{B(0,T)}{B(0,t)}$, l'expression du prix d'une obligation zéro coupon sous le modèle de Ho & Lee est égale à :

$$
B(t,T) =\frac{B(0,T)}{B(0,t)} exp(-{\sigma^2}{2}t(T-t)^2 -X_t (T-t))
$$

### Volatilité de type Hull and White

L'un des inconvénients majeur du modèle de Ho & Lee est l'absence de mécanisme de retour à la moyenne de la volatilité des taux forwards, ce qui peut entraîner une divergence non souhaitable des valeurs de celle ci. C'est ainsi que le modèle de Hull and White permet d'introduire ce mécanisme, en spécifiant la volatilité de cette manière :

$$
\sigma(t,T) = \gamma(t)e^{-\lambda (T-t)}
$$

Dans le cas où la fonction $\gamma(t)$ est constante égale à $\gamma$, alors il s'agit du modèle classique de Hull and White à un facteur. Par simplicité pour la calibration, nous nous placerons dans ce cas de figure pour la suite.

De ce fait :

$$
\Sigma(t,T) =  \int_t^T \sigma(t,u)du = \int_t^T \gamma e^{-\lambda (u-t)} du = \gamma\frac{1-e^{-\lambda (T-t)}}{\lambda}
$$

Et la dynamique du taux forward $f(t,T)$ sous probabilité risque neutre est donnée par :

$$
df(t,T) = \sigma(t,T) \Sigma(t,T)dt + \Sigma(t,T) dW^{\mathbb{Q}}_t = \gamma^2 e^{-\lambda (T-t)} \frac{1-e^{-\lambda (T-t)}}{\lambda}dt +\gamma e^{-\lambda (T-t)} dW^{\mathbb{Q}}_t 
$$

Sachant que le prix à l'instant $t$ d'un zéro coupon de maturité $T$ est donné par $B(t,T) = e^{-\int_t^T f(t,s) ds}$, et que dans le cadre de Hull and White, sa dynamique est donnée par :

$$
dB(t,T) = B(t,T)(r_t dt - \Sigma(t,T) dW^{\mathbb{Q}}_t)
$$

Alors sous le modèle de Hull and White, elle devient :

$$
dB(t,T) = B(t,T)(r_t dt -\gamma\frac{1-e^{-\lambda (T-t)}}{\lambda} dW^{\mathbb{Q}}_t)
$$

Et des calculs non détaillés ici, issus de l'intégration de la dynamique du taux forward $f(t,T)$ et en utilisant la relation $r_t = f(t,t)$ permettent d'établir la dynamique du taux court sous le modèle de Hull and White à un facteur :

$$
dr_t = [\frac{\partial f}{\partial T}(0,t) + \int_0^t \gamma^2 e^{-2\lambda(T-u)}du +\lambda(f(0,t)-r_t)]dt + \gamma dW^{\mathbb{Q}}_t
$$

### Calibration du modèle sur les Caplets

#### Formule de Black sous le cadre de Heath-Jarrow-Morton

Il convient par la suite de calibrer les modèles issus du cadre de Heath-Jarrow-Morton via le prix actuel des caplets. D'après les calculs précédents, l'expression de ce dernier sous la probabilité $T+\delta$ forward neutre est donnée par :

$$
Caplet(T,\delta,K) = \delta B(0,T+\delta) \mathbb{E}^{\mathbb{Q}^{T+\delta}}[(L(T, T+\delta) - K)_{+}]
$$

Or il convient de rappeler que le taux interbancaire $L(T,T+\delta)$ est exprimé en fonction du prix des obligations zéro coupons de la manière suivante :

$$
L(T,T+\delta) = \frac{1}{\delta}(\frac{B(T,T)}{B(T,T+\delta)} -1) = \frac{1}{\delta}(\frac{1}{B(T,T+\delta)} -1)
$$

Ainsi, il est possible de réexprimer le prix d'un Caplet de la manière suivante :

$$
Caplet(T,\delta,K) =  B(0,T+\delta) \mathbb{E}^{\mathbb{Q}^{T+\delta}}[(\frac{B(T,T)}{B(T,T+\delta)} -1 - \delta K)_{+}] =  B(0,T+\delta) \mathbb{E}^{\mathbb{Q}^{T+\delta}}[(\frac{B(T,T)}{B(T,T+\delta)} - (1 + \delta K))_{+}]
$$

De ce fait, en posant $\beta_{T,\delta}(t) = \frac{B(t,T)}{B(t,T+\delta)}$ l'inverse du prix à l'instant $t$ d'un contrat forward sur une obligation zéro coupon prévalent en $T$ et de ténor $\delta$, il est possible d'établir que :

$$
Caplet(T,\delta,K) =  B(0,T+\delta) \mathbb{E}^{\mathbb{Q}^{T+\delta}}[(\beta_{T,\delta}(T) - (1 + \delta K))_{+}]
$$

Il convient de rappeler que le numéraire associé à la probabilité $T+\delta$ forward neutre étant le prix du zéro coupon $B(0,T+\delta)$, le taux forward interbancaire $L_t$ ainsi que le processus $\beta_{T,\delta}(t)$ sont des martingales sous cette probabilité.

En particulier, il est connu que la dynamique selon le cadre de Heath Jarrow Morton d'une obligation zéro coupon sous la probabilité risque neutre est donnée par :

$$
dB(t,T) = B(t,T)(r_t dt - \Sigma(t,T) dW^{\mathbb{Q}}_t)
$$

Par ailleurs, sachant que $\beta_{T,\delta}(t) = \frac{B(t,T)}{B(t,T+\delta)}$, alors un simple calcul d'Itô sur la fonction $f(x,y) = \frac{x}{y}$ avec $x = B(t,T)$ et $y = B(t,T+\delta)$ donne :

$$
d\beta_{T,\delta}(t) = d(\frac{B(t,T)}{B(t,T+\delta)}) = \frac{B(t,T)}{B(t,T+\delta)}[\Sigma(t,T+\delta)(\Sigma(t,T+\delta) - \Sigma(t,T))dt + (\Sigma(t,T+\delta) - \Sigma(t,T))dW^{\mathbb{Q}}_t]
$$

Ainsi, sachant que sous la probabilité $T+\delta$ forward neutre, le mouvement brownien associé à cette probabilité a pour dynamique :

$$
dW^{\mathbb{Q}^{T+\delta}}_t = dW^{\mathbb{Q}}_t+ \Sigma(t,T+\delta)dt
$$ 

le changement de numéraire permet d'établir la dynamique du processus $\beta_{T,\delta}(t)$ sous le modèle HJM de la manière suivante, sous la probabilité $T+\delta$ forward neutre:

$$
d\beta_{T,\delta}(t) = \frac{B(t,T)}{B(t,T+\delta)}[\Sigma(t,T+\delta)(\Sigma(t,T+\delta) - \Sigma(t,T))dt + (\Sigma(t,T+\delta) - \Sigma(t,T))(dW^{\mathbb{Q}^{T+\delta}}_t - \Sigma(t,T+\delta)dt)]
$$

D'où :

$$
d\beta_{T,\delta}(t) = \beta_{T,\delta}(t)(\Sigma(t,T+\delta) - \Sigma(t,T))dW^{\mathbb{Q}^{T+\delta}}_t
$$

Ainsi, l'expression de $\beta_{T,\delta}(T)$ est donnée par :

$$
\beta_{T,\delta}(T)  = \beta_{T,\delta}(0) exp(\int_0^T (\Sigma(t,T+\delta) - \Sigma(t,T))dW^{\mathbb{Q}^{T+\delta}}_t - \frac{1}{2}\int_0^T (\Sigma(t,T+\delta) - \Sigma(t,T))^2 dt) 
$$

Or, sachant que l'intégrale de Wiener $\int_0^T (\Sigma(t,T+\delta) - \Sigma(t,T))dW^{\mathbb{Q}^{T+\delta}}_t$ est une variable aléatoire gaussienne de variance $\frac{1}{2}\int_0^T (\Sigma(t,T+\delta) - \Sigma(t,T))^2 dt$,

alors en posant :

- $V^2(T) = \int_0^T (\Sigma(t,T+\delta) - \Sigma(t,T))^2 dt$
- $Z$ une variable aléatoire centrée réduite 

il est possible de réexprimer $\beta_{T,\delta}(T)$ comme suit, sous la probabilité historique :

$$
\beta_{T,\delta}(T) = \beta_{T,\delta}(0)e^{V(T) Z -\frac{1}{2} V^2(T)}
$$

Et le prix d'un Caplet selon le modèle de Ho & Lee sous cette même probabilité est donné par :

$$
Caplet(T,\delta,K) = B(0,T+\delta) \mathbb{E}[\beta_{T,\delta}(0)(e^{V(T) Z -\frac{1}{2} V^2(T)} - (1 + \delta K))_{+}] =  B(0,T+\delta) \mathbb{E}[\beta_{T,\delta}(0)(e^{V(T) Z -\frac{1}{2} V^2(T)} - (1 + \delta K))_{+}]
$$

Or il a été vu précédemment dans le cadre du pricing selon la formule de Black que pour une volatilité $\sigma(T,\delta,K)$ constante par rapport au temps $t$ mais dépendant de la maturité $T$, du ténor $\delta$ et du strike $K$,

$$
\mathbb{E}[(L_0e^{\sigma(T,\delta,K)\sqrt{T}Z - \frac{1}{2}\sigma^2(T,\delta,K)T} - K)_{+}] = L_0\mathcal{N}(d)- K\mathcal{N}(d-v)
$$

avec $d = \frac{1}{v}(ln(\frac{L_0}{K}) -\frac{v^2}{2})$ et $v =\sigma(T,\delta,K)\sqrt{T}$

Dans le cadre de Heath-Jarrow-Morton, la quantité $\sigma(T,\delta,K)$ est égale à $V(T)$ . Par ailleurs, en remplaçant dans l'expression ci dessus $K$ par $1+\delta K$ et $L_0$ par $\beta_{T,\delta}(0)$, aors il est possible d'obtenir le prix d'un caplet selon le modèle :

$$
Caplet(T,\delta,K) =   B(0,T+\delta) (\beta_{T,\delta}(0)\mathcal{N}(d)- (1+\delta K)\mathcal{N}(d-v))
$$

avec :

$$d = \frac{1}{V^2(T)}(ln(\frac{\beta_{T,\delta}(0)}{1+\delta K}) +\frac{V^2(T)}{2}) = \frac{1}{V^2(T)}(ln(\frac{B(0,T)}{B(0,T+\delta)(1+\delta K)})) +\frac{V^2(T)}{2})
$$ 

et 

$$ 
V^2(T) = \int_0^T (\Sigma(t,T+\delta) - \Sigma(t,T))^2 dt
$$

Ainsi, le prix d'un caplet selon la formule de Black dans le cadre de Heath-Jarrow-Morton peut s'écrire comme le produit du prix actuel $B(0,T+\delta)$ d'un zéro coupon de maturité $T+\delta$ et du prix Black d'un Call avec les paramètres suivant :

- Forward : $\beta_{T,\delta}(0) = \frac{B(0,T)}{B(0,T+\delta)}$
- Strike : $1 +\delta K$
- Maturité : $T$
- volatilité de Black : $\sigma^B = V(T) = \sqrt{\int_0^T (\Sigma(t,T+\delta) - \Sigma(t,T))^2 dt}$

#### Calibration du modèle de Ho & Lee sur les caplets

Par ailleurs sachant que sous le modèle de Ho & Lee, $\Sigma(t,T) = \sigma (T-t)$, alors :

$$
V^2(T) = \int_0^T (\Sigma(t,T+\delta) - \Sigma(t,T))^2 dt = \sigma^2 \int_0^T \delta^2 dt = \sigma^2 \delta^2 T
$$

Sachant que le modèle de Ho & Lee est directement issu du cadre de Heath-Jarrow-Morton avec ici $V^2(T) =\sigma^2 \delta^2 T$, alors d'après le résultat précédent, il est donc possible d'obtenir le prix d'un caplet sous ce modèle :

$$
Caplet(T,\delta,K) =   B(0,T+\delta) (\beta_{T,\delta}(0)\mathcal{N}(d)- (1+\delta K)\mathcal{N}(d-V(T)))
$$

avec $d = \frac{1}{V(T)}(ln(\frac{\beta_{T,\delta}(0)}{1+\delta K}) +\frac{V^2(T)}{2}) = \frac{1}{v}(ln(\frac{B(0,T)}{B(0,T+\delta)(1+\delta K)})) +\frac{V^2(T)}{2})$ avec $V(T) =\sigma \delta\sqrt{T}$

Ainsi, le prix d'un caplet sous le modèle de Ho & Lee peut s'écrire comme le produit du prix actuel $B(0,T+\delta)$ d'un zéro coupon de maturité $T+\delta$ et du prix Black d'un Call avec les paramètres suivant :

- Paramètre Spot : $\beta_{T,\delta}(0) = \frac{B(0,T)}{B(0,T+\delta)}$
- Strike : $1 +\delta K$
- Maturité : $T$
- volatilité Black: $\sigma^B = V(T) = \sigma \delta\sqrt{T}$

## Pricing d'instruments exotiques

### Changement de probabilité

Il a été vu précédemment que le processus $X_t = r_t - f(0,t)$ avait pour dynamique sous la probabilité risque neutre :
  
$$
dX_t = \sigma^2 tdt + \sigma dW^{\mathbb{Q}}_t
$$

Or le numéraire associé à la probabilité $T+\delta$ forward neutre, c'est à dire l'obligation zéro coupon de maturité $T+\delta$, a pour dynamique sous la probabilité risque neutre et sous le modèle de Ho & Lee :

$$
\frac{dB(t,T+\delta)}{B(t,T+\delta)} = r_tdt - \sigma(T+\delta - t)dW^{\mathbb{Q}}_t
$$

Par ailleurs, le mouvement brownien associé à la probabilité $T+\delta$ forward neutre a pour dynamique 

$$
dW^{\mathbb{Q}^{T+\delta}}_t = dW^{\mathbb{Q}}_t+\sigma(T+\delta - t)
$$

Ainsi, sous cette même probabilité, il est naturel que sous le modèle de Ho & Lee, le processus $X_t$ ait pour dynamique :

$$
dX_t = \sigma^2 (2t-T-\delta) dt + \sigma dW^{\mathbb{Q}^{T+\delta}}_t
$$

### Pricing d'une option barrière

Une option barrière est une option délivrant un payoff uniquement si le prix du sous-jacent ait franchi ou non une barrière. Plus particulièrement, dans le cas d'une option Up and Out, le payoff devient nul lorsque le sous-jacent franchit une certaine barrière entre le début et la maturité de l'option.

Plus précisément, dans le cadre de ce projet, il est question de considérer un caplet de type Up and Out, c'est à dire un caplet de Strike ATM, c'est à dire qu'il est égal au prix actuel du sous-jacent qui est le taux linéaire pour des prêts de durée $\delta$, et de Barrière égal à ATM + B, avec B strictement positif. Cette barrière provoque ainsi la désactivation du caplet lorsque le taux linéaire forward de maturité $T$ la franchit à tout moment de la vie du caplet.

Ainsi, le payoff du caplet Up and Out est donné par :

$$
\Pi_{Caplet}(T,\delta,K) = \delta(L(T, T+\delta) - K)_{+}\mathbb{1}_{sup_{s \in [0,T]} L(s,T, T+\delta) < B}
$$

Notons que lorsque la quantité $B$ de la barrière tend vers $+ \infty$, alors le payoff dudit caplet barrière tend vers celui d'un caplet européen classique, impliquant donc le même phénomène pour son prix à chaque instant $t$. Au contraire, si $B$ tend vers $- \infty$, alors le prix du caplet Up and Out à chaque instant tend vers zéro, étant donné qu'il est quasiment impossible d'avoir des taux divergeant négativement.

En l'absence d'opportunité d'arbitrage, si l'on pose $\mathbb{Q}$ la probabilité risque neutre et $r_t$ le taux sans risque, le théorème fondamental de la valorisation des actifs permet d'établir que le prix actuel d'un caplet est donné par :

$$
Caplet_{UO}(T,\delta,K) = \delta \mathbb{E}^{\mathbb{Q}}[e^{\int_0^{T+\delta} r_s ds}(L(T, T+\delta) - K)_{+}\mathbb{1}_{sup_{s \in [0,T]} L(s,T,T+\delta) < K+B}]
$$

De ce fait, en utilisant le même raisonnement que celui pour le pricing des caplets vanilles, le changement de numéraire selon la probabilité $T+\delta$ forward neutre permet d'obtenir le prix suivant :

$$
Caplet_{UO}(T,\delta,K) = \delta B(0,T+\delta) \mathbb{E}^{\mathbb{Q}^{T+\delta}}[(L(T, T+\delta) - K)_{+}\mathbb{1}_{sup_{s \in [0,T]} L(s,T,T+\delta) < K+B}]
$$

Or il convient de rappeler que le taux interbancaire $L(T,T+\delta)$ est exprimé en fonction du prix des obligations zéro coupons de la manière suivante :

$$
L(T,T+\delta) = \frac{1}{\delta}(\frac{B(T,T)}{B(T,T+\delta)} -1) = \frac{1}{\delta}(\frac{1}{B(T,T+\delta)} -1)
$$

Et que le taux forward a pour expression à l'instant $t$ :

$$
L(t,T,T+\delta) = \frac{1}{\delta}(\frac{B(t,T)}{B(t,T+\delta)} -1)
$$

De ce fait, en posant une fois de plus $\beta_{T,\delta}(t) = \frac{B(t,T)}{B(t,T+\delta)}$ l'inverse du prix à l'instant $t$ d'un contrat forward sur une obligation zéro coupon prévalent en $T$ et de ténor $\delta$, il est possible d'établir que :

$$
Caplet_{UO}(T,\delta,K) =  B(0,T+\delta) \mathbb{E}^{\mathbb{Q}^{T+\delta}}[(\beta_{T,\delta}(T) - (1 + \delta K))_{+}\mathbb{1}_{sup_{s \in [0,T]} \beta_{T,\delta}(s) < \delta(K+B)+1}]
$$


#### Valorisation Monte Carlo sous le modèle de Ho & Lee

L'équation différentielle stochastique du processus $\beta_{T,\delta}(t)$ sous la probabilité $\mathbb{Q}^{T+\delta}$ est donné par :

$$
d\beta_{T,\delta}(t) = \beta_{T,\delta}(t) \delta \sigma dW^{\mathbb{Q}^{T+\delta}}_t
$$

Il convient d'estimer le prix des caplets Up and Out par Monte Carlo. Pour cela, sachant que celui ci est caractérisé par l'ensemble de la trajectoire de $\beta_{T,\delta}(t)$, étant donné la présence de la barrière désactivante, il convient de simuler non pas le processus à l'instant final $T$, mais l'ensemble de sa trajectoire entre l'instant $0$ et $T$. Pour cela, l'équation différentielle stochastique est discrétisé selon le Schéma d'Euler, en considérant une discrétisation du temps $(t_0 = 0,...,t_n = T)$ avec $t_i = \Delta i$ pour $i \in [|0,n|]$ et $\Delta = \frac{T}{n}$ le différentiel en pas de temps. 

Sous la probabilité $\mathbb{Q}^{T+\delta}$, la variable $dW^{\mathbb{Q}^{T+\delta}}_t$ est gaussienne de loi $\mathcal{N}(0,dt)$. Ainsi, en posant $Z$ une variable aléatoire centrée réduite, pour $i \in [|0,n|]$, cette discrétisation est donnée par

$$
\beta_{T,\delta}(t_{i+1}) = \beta_{T,\delta}(t_i) \delta \sigma \sqrt{\Delta}Z
$$

Sachant que le prix actuel d'un caplet Up and Out est donné par :

$$
Caplet_{UO}(T,\delta,K) =  B(0,T+\delta) \mathbb{E}^{\mathbb{Q}^{T+\delta}}[(\beta_{T,\delta}(T) - (1 + \delta K))_{+}\mathbb{1}_{sup_{s \in [0,T]} \beta_{T,\delta}(s) < \delta(K+B)+1}]
$$

Alors pour $M$ suffisamment grand pour s'assurer de la convergence de l'estimateur grâce à la Loi Forte des Grands Nombres, son estimateur Monte Carlo est donné par :

$$
\hat{Caplet_{UO}}(T,\delta,K) =  B(0,T+\delta)*\frac{1}{M}\sum_{j=1}^M[(\beta^j_{T,\delta}(T) - (1 + \delta K))_{+}\mathbb{1}_{max_{i \in [|0,n|]} \beta_{T,\delta}(t_i) < \delta(K+B)+1}]
$$

Pour résumer, la simulation Monte Carlo du prix actuel des caplets barrières Up and Out se fait comme suit :

- On simule $M$ variables gaussiennes centrées réduites $(Z_j)$ pour $j \in [|1,M|]$
- On applique le schéma d'Euler sur l'équation différentielle stochastique pour obtenir $\beta^j_{T,\delta}(t_{i+1}) = \beta^j_{T,\delta}(t_i) \delta \sigma \sqrt{\Delta}Z_j$ pour $i \in [|0,n|]$ et pour $j \in [|1,M|]$
- On calcule $\hat{Caplet_{UO}}(T,\delta,K)$ selon la formule ci dessus

En posant la fonction $g$ qui est croissante en fonction du processus $\beta^j_{T,\delta}(T)$ et définie telle que :

$$
g(\beta^j_{T,\delta}(T)) = (\beta^j_{T,\delta}(T) - (1 + \delta K))_{+}\mathbb{1}_{max_{i \in [|0,n|]} \beta^j_{T,\delta}(t_i) < \delta(K+B)+1}
$$

Sa variance est donnée par :

$$
\sigma^2_{Caplet} = \frac{1}{M-1}\sum_{j=1}^M (g(\beta^j_{T,\delta}(T)) - \hat{Caplet_{UO}}(T,\delta,K))^2
$$

Ce qui permet d'en établir un intervalle de confiance à 95% :

$$
IC_{95} = [Caplet_{UO}(T,\delta,K) - \frac{\sigma_{Caplet}}{\sqrt{M}}, Caplet_{UO}(T,\delta,K) + \frac{\sigma_{Caplet}}{\sqrt{M}}]
$$.

##### Méthode des variables antithétiques

Un des inconvénients majeurs de l'estimation traditionnelle de Monte Carlo telle qu'elle est définie ci dessus est la possibilité d'obtenir une variance trop élevée. Plus particulièrement, lorsque l'on simule une multitude de trajectoires de mouvements browniens standards, il est possible de constater qu'à l'instant final $T$, la variance de l'ensemble des variables $W^j_t$ est trop élevée, ce qui pose problème pour la stabilité des estimateurs.

C'est pourquoi des méthodes de réduction de variances sont utilisés pour effectuer les simulations de Monte Carlo. L'une des plus connues est la méthode des variables antithétiques, qui sera ici appliquée. Le raisonnement est expliqué comme suit :

Il convient dans un premier temps de remarquer la symétrie en loi des mouvements browniens standards, qui en est une propriété primordiale. En effet, les processus $(W_t)$ et $(-W_t)$ sont de même loi $\mathcal{N}(0,t)$. Par ailleurs, leur corrélation est négative ce qui a un effet négatif sur la variance de l'estimateur de Monte Carlo. 

En effet, si l'on souhaite estimer $I = \mathbb{E}[g(W_T)]$, $g$ étant une fonction mesurable du mouvement brownien en $T$, il est possible dans un premier temps de remarquer que $I = \frac{1}{2}(\mathbb{E}[g(W_T)] + \mathbb{E}[g(-W_T)])$. De ce fait, un estimateur par la méthode des variables antithétiques est donné apr :

$$
\tilde{I} = \frac{1}{M}\sum_{j=1}^M \frac{g(W^j_T) + g(-W^j_T)}{2}
$$

La variance d'un tel estimateur fait intervenir la covariance du mouvement brownien et son opposé. La négativité de cette dernière permet ainsi une variance plus faible de $\tilde{I}$ que celle de l'estimateur traditionnelle de Monte Carlo.

Appliqué dans le cas présent, la méthode des variables antithétiques peut être détaillée comme suit :

- On simule $M$ variables gaussiennes centrées réduites $(Z_j)$ pour $j \in [|1,M|]$
- On applique le schéma d'Euler sur l'équation différentielle stochastique pour obtenir $\beta^j_{T,\delta}(t_{i+1}) = \beta^j_{T,\delta}(t_i) \delta \sigma \sqrt{\Delta}Z_j$ pour $i \in [|0,n|]$ et pour $j \in [|1,M|]$
- On considère ensuite l'opposé de l'échantillon $(Z_1,...,Z_M)$, qui est donc $(-Z_1,...,-Z_M)$
- Le schéma d'Euler est ensuite réappliqué pour obtenir $\beta^{'j}_{T,\delta}(t_{i+1}) = -\beta^{'j}_{T,\delta}(t_i) \delta \sigma \sqrt{\Delta}Z_j$ pour $i \in [|0,n|]$ et pour $j \in [|1,M|]$

Et l'estimateur par variables antithétiques du prix actuel d'un Caplet Up and Out est donnée par :

$$
\tilde{Caplet_{UO}}(T,\delta,K) =  B(0,T+\delta)*\frac{1}{M}\sum_{j=1}^M \frac{g(\beta^j_{T,\delta}(T)) + g(\beta^{'j}_{T,\delta}(T))}{2}
$$

avec :

$$
g(\beta^j_{T,\delta}(T)) = (\beta^j_{T,\delta}(T) - (1 + \delta K))_{+}\mathbb{1}_{max_{i \in [|0,n|]} \beta^j_{T,\delta}(t_i) < \delta(K+B)+1}
$$

Cet estimateur converge vers le prix réel des caplets grâce à la loi des grands nombres, mais possède une variance réduite par rapport à l'estimateur traditionnel de Monte Carlo.

### Pricing d'une obligation avec une clause de rappel

### Simulation de Monte Carlo